In [1]:
import cv2 as cv
import math
import numpy as np
##角度制转弧度制
RAD = lambda v: v*math.pi/180;
##xyz方向上偏转矩阵
Rx = lambda rx: np.mat([[1,0,0],[0,math.cos(rx),-math.sin(rx)],[0,math.sin(rx),math.cos(rx)]])
Ry = lambda ry: np.mat([[math.cos(ry),0,math.sin(ry)],[0,1,0],[-math.sin(ry),0,math.cos(ry)]])
Rz = lambda rz: np.mat([[math.cos(rz),-math.sin(rz),0],[math.sin(rz),math.cos(rz),0],[0,0,1]])

print(RAD(60))
Rx(1.1)

1.0471975511965976


matrix([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.45359612, -0.89120736],
        [ 0.        ,  0.89120736,  0.45359612]])

In [2]:
flag = 0
point=[0,1]
def OnMouseAction(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDOWN:
        if flag == 0:
            point[0]=np.mat([x,y]).T
#             print("第一次单击",point[0])
        elif flag == 1:
            point[1]=np.mat([x,y]).T
#             print("第二次单机",point[1])
            
if __name__=="__main__":
    ##参数
    inputfile= "./data/daejeon_station.png"
    f,cx,cy,L=810.5,480,270,3.31
    cam_ori=np.mat([RAD(-18.7),RAD(-8.2),RAD(2.0)])
    grid_x,grid_z=range(-2,3),range(5,35)
    image=cv.imread(inputfile)
    
    ##绑定image图像，获取到headpoint和concatpoint
    cv.namedWindow('image')
    cv.setMouseCallback('image',OnMouseAction)
    while(1):
        cv.imshow('image',image)
        k=cv.waitKey(1)
        if k==ord('z'):
            flag = 1
        elif k==ord('q'):
            break
    print("concat point: ",point[0],"\nhead point: ",point[1])
    start,end=point[0],point[1]##没有point所以使用列矩阵表示点坐标
    cv.destroyAllWindows()
    
    ##绘制网格，绘制到image上，所以需要有x’转换成x：x'=Rx+t & x=R'x'+t'-->R=R^-1=R'^T,t=-R'^T*t'
    K=np.mat([[f,0,cx],[0,f,cy],[0,0,1]])#camera matrix
    Rc=Rz(cam_ori[0,2])*Ry(cam_ori[0,1])*Rx(cam_ori[0,0])#roration matrix
    R=Rc.T##Rc是正交矩阵，其Rc的逆即是其转置
    tc=np.mat([0,-L,0]).T##公式
    t=-R*tc##公式
    for z in grid_z:
        p=K*(R*np.mat([grid_x[0],0,z]).T+t)
        q=K*(R*np.mat([grid_x[-1],0,z]).T+t)
        cv.line(image,(int(p[0,0]/p[2,0]),int(p[1,0]/p[2,0])),(int(q[0,0]/q[2,0]),int(q[1,0]/q[2,0])),(64, 128, 64),1)
    for x in grid_x:
        p=K*(R*np.mat([x,0,grid_z[0]]).T+t)
        q=K*(R*np.mat([x,0,grid_z[-1]]).T+t)
        cv.line(image,(int(p[0,0]/p[2,0]),int(p[1,0]/p[2,0])),(int(q[0,0]/q[2,0]),int(q[1,0]/q[2,0])),(64, 128, 64),1)
  
#     cv.imshow('image',image)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    print("start---",start)
    print("end---",end)
    
    while(1):
        img=image.copy()
        if end[0,0]>0 and end[1,0]>0: 
            ##计算Z、H、X值
            cp=R.T*np.mat([start[0,0]-cx,start[1,0]-cy,f]).T
            if cp[1,0]<0.001:##剔除接触点位置超出的情况
                continue
            hp=R.T*np.mat([end[0,0]-cx,end[1,0]-cy,f]).T
            Z=cp[2,0]/cp[1,0]*L
#             X=cp[0,0]/cp[1,0]*L
            H=(cp[1,0]/cp[2,0]-hp[1,0]/hp[2,0])*Z
            print("Z:",Z,"H",H)
            
            cv.circle(img,(start[0,0],start[1,0]),4,(255,0,0),-1)
            cv.circle(img,(end[0,0],end[1,0]),4,(255,0,0),-1)
            #图像/文字/位置/字体/字体大小/颜色/字体粗细
            cv.putText(img,"Z:{:.3},H:{:.3}".format(Z,H),(start[0,0]+10,start[1,0]+10),
                                                          cv.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
            cv.imshow("image_copy",img)
            key=cv.waitKey(0)
            if key==ord('q'):
                break
    cv.destroyAllWindows()

concat point:  [[606]
 [246]] 
head point:  1
start--- [[606]
 [246]]
end--- 1


TypeError: 'int' object is not subscriptable